# PySpark with Titanic Data

---

NOTE: The first cell below containing `SparkContext` initialization does not need to run if you have PySpark installed locally and configured to launch with an Anaconda virtual environment. 

In [79]:
# from pyspark import SparkContext
# sc = SparkContext()

In [94]:
from pyspark.sql import SparkSession

SPARK_APP_NAME = "Python Spark Titanic Data Investigation"
SPARK_CONFIG_OPT, SPARK_CONFIG_VAL = "spark.some.config.option", "some-value"

spark = SparkSession.builder.appName(SPARK_APP_NAME).config(SPARK_CONFIG_OPT, SPARK_CONFIG_VAL).getOrCreate()

In [95]:
FILENAME = "titanic.csv"

titanicDF = spark.read.csv(FILENAME, header=True, inferSchema=True)

In [96]:
# titanicDF.first()
# titanicDF.take(3)
titanicDF.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [97]:
titanicDF.describe().show(3)

+-------+-----------------+-------------------+------------------+----+----+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|Name| Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+----+----+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891| 891| 891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|null|null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.2042079685746| null|    null|
| stddev|257.3538420152301|0.48659245426485753|0.8360712409770491

## What is the percentage of female passengers younger than 55 years of age?

In [98]:
q1_df = titanicDF.groupBy(titanicDF.Age < 55).count()

total = q1_df.select("count").agg({"count": "sum"}).collect().pop()["sum(count)"]
result = q1_df.withColumn("percent (%)", (q1_df["count"] / total) * 100)

result.show()

+----------+-----+------------------+
|(Age < 55)|count|       percent (%)|
+----------+-----+------------------+
|      null|  177|19.865319865319865|
|      true|  672| 75.42087542087542|
|     false|   42| 4.713804713804714|
+----------+-----+------------------+



## Discretize the `Age` continuous values into a new column containing categories for _Children_ (0 - 10y), _Young_ (10 - 50y), and _Old_ (> 50y).

NOTE: In order to easily discretize, we will naïvely assume that null ages are zero (fall under _Children_). 

In [99]:
titanicDF = titanicDF.na.fill(0)

In [104]:
AGE_PARAM_CHILDREN = (titanicDF.Age < 10)
AGE_PARAM_YOUNG = ((titanicDF.Age >= 10) & (titanicDF.Age <= 50))
AGE_PARAM_OLD = (titanicDF.Age > 50)

titanicDF.groupBy(AGE_PARAM_CHILDREN, AGE_PARAM_YOUNG, AGE_PARAM_OLD).count().show()

+----------+-----------------------------+----------+-----+
|(Age < 10)|((Age >= 10) AND (Age <= 50))|(Age > 50)|count|
+----------+-----------------------------+----------+-----+
|      true|                        false|     false|  239|
|     false|                         true|     false|  588|
|     false|                        false|      true|   64|
+----------+-----------------------------+----------+-----+



In [105]:
from pyspark.ml.feature import Bucketizer

bucketizer = Bucketizer(splits=[0, 10, 50, float("Inf") ],inputCol="Age", outputCol="buckets")
titanicDF_bucketed = bucketizer.setHandleInvalid("keep").transform(titanicDF)

titanicDF_bucketed.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|buckets|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|    1.0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|    1.0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|    1.0|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
only showing top 3 rows



In [117]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

age_map = {0.0: "CHILDREN", 1.0: "YOUNG", 2.0: "ADULT"}
udf_titanic = udf(lambda X: age_map[X], StringType())
titanicDF_bucketed = titanicDF_bucketed.withColumn("age_group", udf_titanic("buckets"))

titanicDF_bucketed.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+---------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|buckets|age_group|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+---------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|    1.0|    YOUNG|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|    1.0|    YOUNG|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|    1.0|    YOUNG|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|    1.0|    YOUNG|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0| 

## Find the percentage of each category across the data.

In [130]:
CHILDREN_BUCKET = (titanicDF_bucketed.age_group == "CHILDREN")
YOUNG_BUCKET = (titanicDF_bucketed.age_group == "YOUNG")
OLD_BUCKET = (titanicDF_bucketed.age_group == "OLD")

q2_df = titanicDF_bucketed.groupBy(CHILDREN_BUCKET, YOUNG_BUCKET, OLD_BUCKET).count()

total = q2_df.select("count").agg({"count": "sum"}).collect().pop()["sum(count)"]
result = q2_df.withColumn("percent (%)", (q2_df["count"] / total) * 100)

result.show()

+----------------------+-------------------+-----------------+-----+------------------+
|(age_group = CHILDREN)|(age_group = YOUNG)|(age_group = OLD)|count|       percent (%)|
+----------------------+-------------------+-----------------+-----+------------------+
|                 false|              false|            false|   74| 8.305274971941639|
|                  true|              false|            false|  239|26.823793490460158|
|                 false|               true|            false|  578| 64.87093153759821|
+----------------------+-------------------+-----------------+-----+------------------+

